In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!pip install fastapi uvicorn pyngrok efficientnet_pytorch torchvision pillow requests


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.7 MB/s eta 0:00:00
   ━━

In [ ]:
from pyngrok import ngrok

ngrok.set_auth_token('2vyGA7Tyh2LoFJeFRO17NUXS38Z_5qEBYN7jcEn4LTgH6cvk5')


In [ ]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet

# Load EfficientNet-B2 model
model = EfficientNet.from_pretrained('efficientnet-b2')

# Modify the last layer
model._fc = nn.Linear(model._fc.in_features, 1)

# Load your trained weights
model.load_state_dict(torch.load('/content/drive/MyDrive/eye_dataset/trained_model/trained_model_1.pth', map_location=torch.device('cpu')))

# Set model to evaluation mode
model.eval()


Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b2-8bb594d6.pth" to /root/.cache/torch/hub/checkpoints/efficientnet-b2-8bb594d6.pth
100%|██████████| 35.1M/35.1M [00:00<00:00, 96.2MB/s]


Loaded pretrained weights for efficientnet-b2


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [ ]:
import torch
import torch.nn as nn
from efficientnet_pytorch import EfficientNet
from fastapi import FastAPI
from pydantic import BaseModel
from PIL import Image
import io
import os
import requests
import uvicorn
from pyngrok import ngrok
from threading import Thread


In [ ]:
from torchvision import transforms

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((260, 260)),  # EfficientNet-B2 expects 260x260
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225]),
])


In [ ]:
# Initialize FastAPI app
app = FastAPI()

# Define request body model
class ImageURL(BaseModel):
    url: str

# Function to download image and save locally
def download_image(image_url, save_path="downloaded_image.jpg"):
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(save_path, "wb") as f:
            f.write(response.content)
        return save_path
    else:
        raise Exception(f"Failed to download image. Status code: {response.status_code}")

# Function to predict disease risk from image
def predict_image(image_path):
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0)  # Add batch dimension

    with torch.no_grad():
        output = model(input_tensor)
        probability = torch.sigmoid(output).item()

    predicted_class = int(probability > 0.5)

    return probability, predicted_class


In [ ]:
@app.post("/predict/")
async def predict(image_data: ImageURL):
    try:
        # Step 1: Download image
        image_path = download_image(image_data.url, save_path="downloaded_image.jpg")

        # Step 2: Predict
        probability, prediction = predict_image(image_path)

        # Step 3: Delete the downloaded image
        os.remove(image_path)

        return {
            "probability": probability,
            "prediction": prediction
        }
    except Exception as e:
        return {"error": str(e)}


In [ ]:
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from threading import Thread
from pyngrok import ngrok

app = FastAPI()

# Allow all CORS origins
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Allow all origins
    allow_credentials=True,
    allow_methods=["*"],  # Allow all HTTP methods
    allow_headers=["*"],  # Allow all headers
)

# Run the FastAPI server
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Start server in a background thread
Thread(target=run, daemon=True).start()

# Expose the port via ngrok and print the public URL
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

# Keep the server running
while True:
    pass


INFO:     Started server process [563]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Public URL: NgrokTunnel: "https://b3f5-34-169-171-95.ngrok-free.app" -> "http://localhost:8000"
INFO:     2405:acc0:1207:3187:b5b8:2e3c:bf91:5dbb:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2405:acc0:1207:3187:b5b8:2e3c:bf91:5dbb:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2405:acc0:1207:3187:b5b8:2e3c:bf91:5dbb:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2405:acc0:1207:3187:b5b8:2e3c:bf91:5dbb:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2405:acc0:1207:3187:b5b8:2e3c:bf91:5dbb:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2405:acc0:1207:3187:b5b8:2e3c:bf91:5dbb:0 - "GET /docs HTTP/1.1" 200 OK
INFO:     2405:acc0:1207:3187:b5b8:2e3c:bf91:5dbb:0 - "GET /openapi.json HTTP/1.1" 200 OK
